In [7]:
import requests
from bs4 import BeautifulSoup

URL = "https://careers.ti.com/en/sites/CX/jobs"
response = requests.get(URL)
soup = BeautifulSoup(response.text, "html.parser")

jobs = []

# TRY job-list-item__job-info-item
# for job_elem in soup.find_all(class_="job-list-item__job-info-label--posting-date"):
for job_elem in soup.find_all(class_="job-list-item__job-info-item"):
    # title = job_elem.select_one(".job-title").text
    # link = job_elem.select_one("a")["href"]
    # jobs.append({"title": title, "link": link})
    print("HELLO")
    jobs.append(job_elem.text)


In [8]:
print("JOBS arr=")
print(jobs)

JOBS arr=
[]


In [30]:
import asyncio
from playwright.async_api import async_playwright
from datetime import date

URL = "https://careers.ti.com/en/sites/CX/jobs"
print(URL)

MAX_ITERS = 1

async def main():
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        await page.goto(URL)
        print(await page.title())

        shadow_page = page.locator("plasmo-csui#jobright-helper-plugin")
        print("shadow_page=", shadow_page)

        frame = page.frame_locator("iframe")
        print("frame=", frame)
        
        await page.goto(URL, wait_until="networkidle")
        await page.wait_for_selector(".app__subpage.app__subpage--custom-page")
        # await page.wait_for_selector("li.job-list-item__job-info-item")
        print("AFTER waiting for li.job-list-item__job-info-item to appear / be loaded on the page")

        linkContainers = page.locator(".app__subpage.app__subpage--custom-page")
        linkContainers = await linkContainers.all()
        for linkContainer in linkContainers:
            print("linkContainer=", linkContainer)
        
        print("FINDING THE PARENT search-job-results CONTAINER")
        linkContainers = page.locator(".search-job-results")
        linkContainers = await linkContainers.all()
        for linkContainer in linkContainers:
            print("linkContainer=", linkContainer)
            
        print("FINDING THE INDIVIDUAL DATE OF A JOB POSTING")
        linkContainers = page.locator("li.job-list-item__job-info-item")
        linkContainers_num = await linkContainers.count()
        print("NUMBER OF job-list-item__job-info-item linkContainers=", linkContainers_num)
        # linkContainers = await linkContainers.all()
        # for linkContainer in linkContainers:
        #     print("\n\nlinkContainer=", linkContainer)
        #     linkContainerHTML = await linkContainer.evaluate("element => element.innerHTML")
        #     print(linkContainerHTML)

        # linkContainers = page.locator("div.job-list-item__job-info-label.job-list-item__job-info-label--posting-date") # Found 25 of these
        # linkContainers = page.locator("div.job-list-item__job-info-value-container > div.job-list-item__job-info-value")
        # Found 50 of these (because it also finds the Job's Location <posting-locations elements)
        
        linkContainers = page.locator("div.job-list-item__job-info-label.job-list-item__job-info-label--posting-date + div div.job-list-item__job-info-value")
        
        linkContainers_num = await linkContainers.count()
        print("NUMBER OF ...posting-date linkContainers=", linkContainers_num)
        linkContainers = await linkContainers.all()
        
        for linkContainer in linkContainers:
            print("\n\nlinkContainer=", linkContainer)
            linkContainerHTML = await linkContainer.evaluate("element => element.innerHTML")
            print(linkContainerHTML)

       
        await browser.close()

await main()

https://careers.ti.com/en/sites/CX/jobs
Search Jobs - Texas Instruments Careers
shadow_page= <Locator frame=<Frame name= url='https://careers.ti.com/en/sites/CX/jobs'> selector='plasmo-csui#jobright-helper-plugin'>
frame= <FrameLocator frame=<Frame name= url='https://careers.ti.com/en/sites/CX/jobs'> selector='iframe'>
AFTER waiting for li.job-list-item__job-info-item to appear / be loaded on the page
linkContainer= <Locator frame=<Frame name= url='https://careers.ti.com/en/sites/CX/jobs'> selector='.app__subpage.app__subpage--custom-page >> nth=0'>
FINDING THE PARENT search-job-results CONTAINER
linkContainer= <Locator frame=<Frame name= url='https://careers.ti.com/en/sites/CX/jobs'> selector='.search-job-results >> nth=0'>
FINDING THE INDIVIDUAL DATE OF A JOB POSTING
NUMBER OF job-list-item__job-info-item linkContainers= 50
NUMBER OF ...posting-date linkContainers= 25


linkContainer= <Locator frame=<Frame name= url='https://careers.ti.com/en/sites/CX/jobs'> selector='div.job-list-it